# Imports and Installs

In [2]:
import numpy as np
import pandas as pd
import sys
import random
import os
random.seed(42)
np.random.seed(42)


In [3]:
!sudo apt-get update
!sudo apt-get install ncbi-blast+

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,310 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,267 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/uni

In [4]:
!pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [biopython]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# This Colab notebook uses the BLAST+ command-line tools to detect a suspicious junction in a sequencing read.

import subprocess
import os
import pandas as pd

# Constants
BLAST_DB_NAME = 'viral_db'
BLAST_OUTPUT_PATH = 'blast_results.tsv'
BLAST_DB_TYPE = 'nucl'
EVALUE_THRESHOLD = '10'

# --- Installation and Database Setup ---

def setup_blast_and_database(viral_genome_path, db_name):
    """
    Installs BLAST+ and creates a BLAST database from a viral genome.

    Args:
        viral_genome_path (str): The path to the multi-FASTA file of viral genomes.
        db_name (str): The name for the new BLAST database.
    """
    print("--- Setting up BLAST+ and database ---")

    # Check if BLAST+ is installed by running `blastn -version`.
    try:
        subprocess.run(["blastn", "-version"], check=True, capture_output=True)
        print("BLAST+ is already installed.")
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("BLAST+ not found. Installing now...")
        # Use conda to install BLAST+
        try:
            # Install Miniconda first if it's not present
            if not os.path.exists("miniconda"):
                print("Miniconda not found. Installing...")
                subprocess.run(
                    "wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh",
                    shell=True, check=True
                )
                subprocess.run("bash miniconda.sh -b -p miniconda", shell=True, check=True)
                os.environ["PATH"] = f"{os.getcwd()}/miniconda/bin:{os.environ['PATH']}"
                print("Miniconda installed.")

            # Install BLAST+ from bioconda channel
            subprocess.run(
                "conda install -c bioconda -c conda-forge blast --yes",
                shell=True, check=True
            )
            print("BLAST+ installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error during BLAST+ installation: {e.stderr.decode()}")
            return

    # Create the BLAST database.
    print(f"Creating BLAST database '{db_name}' from {viral_genome_path}...")
    try:
        command = ["makeblastdb", "-in", viral_genome_path, "-dbtype", BLAST_DB_TYPE, "-out", db_name]
        subprocess.run(command, check=True, capture_output=True)
        print("BLAST database created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error creating BLAST database: {e.stderr.decode()}")

def write_fasta_file(sequences, file_path):
    """Write sequences to a FASTA file from a list."""
    with open(file_path, 'w') as f:
        for i, sequence in enumerate(sequences):
            f.write(f'>{i}\n')
            f.write(f'{sequence}\n')

def run_blast(query_path, db_name, output_path, evalue):
    """Run BLAST and save the results to an output file."""
    subprocess.run([
        'blastn',
        '-query', query_path,
        '-db', db_name,
        '-out', output_path,
        '-outfmt', '6',  # Tabular output
        '-max_hsps', '1',
        '-evalue', evalue,
        '-max_target_seqs', '1',
        '-word_size', '7' # A smaller word size to find matches in simple sequences
    ], check=True)

def parse_blast_results(blast_output_path):
    """Parse BLAST tabular results and return a DataFrame."""
    columns = [
        "query_id", "subject_id", "% identity", "alignment_length", "mismatches",
        "gap_openings", "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"
    ]
    if os.path.getsize(blast_output_path) == 0:
        return pd.DataFrame(columns=columns)

    blast_df = pd.read_csv(blast_output_path, sep='\t', header=None, names=columns)
    return blast_df

def detect_suspicious_junction(df, min_non_matching_bases=30):
    """
    Identifies suspicious reads in a BLAST results DataFrame.
    """
    suspicious_reads = []

    # We need to know the original read length. Since we can't get it from BLAST output alone,
    # we'll use a hardcoded value for this dummy example. In a real pipeline,
    # you would pass in the read lengths from your FASTQ file.
    read_lengths = {
        0: 200, # This is a dummy value for the length of our natural read
        1: 200, # This is a dummy value for the length of our chimeric read
    }

    if df.empty:
      return suspicious_reads

    for index, row in df.iterrows():
        query_id = int(row['query_id'])
        alignment_len = row['alignment_length']
        identity = row['% identity']

        print(f"Query ID: {query_id}, Alignment Length: {alignment_len}, Identity: {identity}")

        # Get the original read length from our lookup.
        original_read_len = read_lengths.get(query_id, 0)

        if original_read_len > 0:
            non_matching_part = original_read_len - alignment_len

            # Apply the criteria from the article.
            if identity > 95 and non_matching_part > min_non_matching_bases:
                suspicious_reads.append(query_id)

    return suspicious_reads

# --- Dummy Input Data and Execution ---
if __name__ == "__main__":
    # Dummy data
    viral_genome_file = "partial_genome.fasta"
    partial_viral_genome = "GATTGTGAGCGATTTGCGTGCGTGCATCCCGCTTCACTGATCTCTTGTTAGATCTTTTTGTAATCTAAACTTTATAAAAACATCCACTCCCTGTAATCTATGCTTGTGGGCGTAGATTTTTCATAGTGGTGTTTATATTCATTTCTGCTGTTAACAGCTTTCAGCCAGGGACGTGTTGTATCCTAGGCAGTGGCCCGCCCATAGGTCACAATGTCGAAGATCAACAAATACGGTCTCGAACTACACTGGGCTCCAGAATTTCCATGGATGTTTGAGGACGCAGAGGAGAAGTTGGATAACCCTAGTAGTTCAGAGGTGGATATGATTTGCTCCACCACTGCGCAAAAGCTGGAAACAGACGGAATTTGTCCTGAAAATCATGTGATGGTGGATTGTCGCCGACTTCTTAAACAAGAGTGTTGTGTGCAGTCTAGCCTAATACGTGAAATTGTTATGAATGCAAGTCCATATGATTTGGAGGTGCTACTTCAAGATGCTT"

    with open(viral_genome_file, "w") as f:
        f.write(">partial_virus\n")
        f.write(partial_viral_genome + "\n")

    setup_blast_and_database(viral_genome_file, BLAST_DB_NAME)

    # Example 1: A natural, non-chimeric read.
    natural_read = partial_viral_genome[100:300]

    # Example 2: A chimeric read with an engineered insert.
    chimeric_read = partial_viral_genome[100:260] + ("T" * 40)

    # Write dummy reads to a query FASTA file.
    query_sequences = [natural_read, chimeric_read]
    query_fasta_file = "query_reads.fasta"
    write_fasta_file(query_sequences, query_fasta_file)

    print("\n--- Running BLAST ---")
    run_blast(query_fasta_file, BLAST_DB_NAME, BLAST_OUTPUT_PATH, EVALUE_THRESHOLD)

    print("\n--- Parsing BLAST Results and Detecting Suspicious Reads ---")
    blast_df = parse_blast_results(BLAST_OUTPUT_PATH)

    if not blast_df.empty:
      print("BLAST results:")
      print(blast_df)

      suspicious_reads = detect_suspicious_junction(blast_df)

      print("\n--- Final Results ---")
      if suspicious_reads:
          print("The following query IDs are suspicious:")
          for read_id in suspicious_reads:
              print(f"  - Read ID: {read_id}")
      else:
          print("No suspicious reads were detected.")

    else:
      print("No BLAST results were found.")


--- Setting up BLAST+ and database ---
BLAST+ is already installed.
Creating BLAST database 'viral_db' from partial_genome.fasta...
BLAST database created successfully.

--- Running BLAST ---

--- Parsing BLAST Results and Detecting Suspicious Reads ---
BLAST results:
   query_id     subject_id  % identity  alignment_length  mismatches  \
0         0  partial_virus       100.0               200           0   
1         1  partial_virus       100.0               161           0   

   gap_openings  q_start  q_end  s_start  s_end         evalue  bit_score  
0             0        1    200      101    300  2.830000e-107        370  
1             0        1    161      101    261   1.350000e-85        298  
Query ID: 0, Alignment Length: 200, Identity: 100.0
Query ID: 1, Alignment Length: 161, Identity: 100.0

--- Final Results ---
The following query IDs are suspicious:
  - Read ID: 1
